In [1]:
from pathlib import Path
import datetime as dt
import pandas as pd

In [2]:
# Function for later use: To Save the DataFrame to a CSV file
def save_csv_file(df, file_path):
    """ 
    Save a DataFrame to a CSV file at the specified file path.

    Parameters:
    - df: DataFrame to save
    - file_path: Path to save the CSV file
    """
    
    # Check if the parent directory exists
    if not file_path.parent.exists():
        print(f"Error: The directory `{file_path.parent}` does not exist.")
        return
    
    if file_path.exists():
        print(f"File `{file_path.name}` already exists. Overwriting file.")
        file_path.unlink()
    
    # Save the DataFrame to the specified file path
    df.to_csv(file_path, index=False)
    print(f"File saved as `{file_path.name}`")

In [3]:
# Function fetches the data from the URL and returns a Dataframe of the vehicle registration data from the specified year
# Note: Data from website is only available from 2016 to 2022
def fetch_ev_data_by_year(year):

    # Available years for data retrieval from afdc.energy.gov
    available_years = range(2016, 2023)

    # Check if the year is within the available range
    if year not in available_years:
        raise ValueError(f"Year {year} is not supported. Please choose a year between {min(available_years)} and {max(available_years)}.")

    # URL for fetching the vehicle registration data
    url = f"https://afdc.energy.gov/vehicle-registration?year={year}"

    try:
        # Attempt to fetch the table data from the URL
        tables = pd.read_html(url)
        df = pd.DataFrame(tables[0])

    except Exception as error:
        # Print the error message if the data fetching fails
        print(f"Error fetching data for year {year}: {error}")
        return None

    # Remove the multi-level column index tuple
    df.columns = df.columns.droplevel(0)

    # Map the original column names to the new column names
    column_map = {
        "State": "state",
        "Electric (EV)": "electric",
        "Plug-In Hybrid Electric (PHEV)": "phev",
        "Hybrid Electric (HEV)": "hev",
        "Biodiesel": "biodiesel",
        "Ethanol/Flex (E85)": "ethanol",
        "Compressed Natural Gas (CNG)": "cng",
        "Propane": "propane",
        "Hydrogen": "hydrogen",
        "Methanol": "methanol",
        "Gasoline": "gasoline",
        "Diesel": "diesel",
        "Unknown Fuel": "unknown"
    }

    # Rename the columns using the column_map
    df.rename(columns=column_map, inplace=True)

    # Add a new column for the year
    df['year'] = year

    # Reorder columns for clarity (having year and state as the first columns)
    cols_to_order = ['year', 'state']
    new_columns = cols_to_order + [col for col in df.columns if col not in cols_to_order]

    # Create a new DataFrame with the reordered columns
    df = df[new_columns]

    return df

In [4]:
# Obtain the DataFrame for the years 2016 to 2022 and concatenate them into a single DataFrame
# Define the range of years to fetch data for
start_year = 2016
end_year = 2022

# Create an empty list to store the DataFrames for each year
# Later, we will concatenate them into a single DataFrame
df_list = []

# Fetch data for each year and append to the list
for year in range(start_year, end_year + 1):

    # Use the function we created to fetch the data for the year
    registrations_year_df = fetch_ev_data_by_year(year)

    # Handle potential fetch errors
    if registrations_year_df is not None:  
        df_list.append(registrations_year_df)

# Concatenate all DataFrames into a single DataFrame
df = pd.concat(df_list, ignore_index=True)

print("Shape:", df.shape)
display(df)

Shape: (364, 14)


,year,state,electric,phev,hev,biodiesel,ethanol,cng,propane,hydrogen,methanol,gasoline,diesel,unknown
0,2016,Alabama,500,900,29100,0,428300,20100,0,0,0,3777300,126500,53900
1,2016,Alaska,200,200,5000,0,55700,4900,0,0,0,525900,44800,19400
2,2016,Arizona,4700,4400,89600,0,427300,17500,0,0,100,4805000,179500,112800
3,2016,Arkansas,200,500,19100,0,320500,12600,0,0,0,2097800,96800,22200
4,2016,California,141500,116700,966700,0,1322600,80600,0,1300,400,27241000,710400,115500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359,2022,Washington,104100,31400,270200,67500,348300,100,100,0,0,5650700,277400,52700
360,2022,West Virginia,1900,1400,18300,15600,127500,100,0,0,0,1267500,45700,10900
361,2022,Wisconsin,15700,10000,105200,46500,549700,300,0,0,0,4577400,144500,26900
362,2022,Wyoming,800,600,6900,19100,58600,0,0,0,0,483000,61200,14200


In [5]:
# Delete the rows with the state value "United States"
df = df[df['state'] != "United States"]

print("Shape:", df.shape)
df['state'].sort_values().unique()

Shape: (357, 14)


array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [6]:
# Save individual state data to CSV files by looping through the unique states
# in the DataFrame and saving each state's data to a separate CSV file

# Get the values from the 'state' column and convert them to a list
states = df['state'].sort_values().unique().tolist()

# Loop through the created `states` list
for state in states: 
    # Create a new DataFrame for each
    states_df = df[df['state'] == state]

    # Reset the index of the DataFrame
    states_df.reset_index(drop=True, inplace=True)

    # Sort the DataFrame by year, electric, and gasoline columns
    states_df.sort_values(by=["year", "electric", "gasoline"], inplace=True)

    # Save the DataFrame to a CSV file
    file_name = f"{state.lower()}_veh_registrations.csv"
    file_path = Path(f"../../../../data/processed_data/veh_registrations/{file_name}")
    save_csv_file(states_df, file_path)

File `alabama_veh_registrations.csv` already exists. Overwriting file.
File saved as `alabama_veh_registrations.csv`
File `alaska_veh_registrations.csv` already exists. Overwriting file.
File saved as `alaska_veh_registrations.csv`
File `arizona_veh_registrations.csv` already exists. Overwriting file.
File saved as `arizona_veh_registrations.csv`
File `arkansas_veh_registrations.csv` already exists. Overwriting file.
File saved as `arkansas_veh_registrations.csv`
File `california_veh_registrations.csv` already exists. Overwriting file.
File saved as `california_veh_registrations.csv`
File `colorado_veh_registrations.csv` already exists. Overwriting file.
File saved as `colorado_veh_registrations.csv`
File `connecticut_veh_registrations.csv` already exists. Overwriting file.
File saved as `connecticut_veh_registrations.csv`
File `delaware_veh_registrations.csv` already exists. Overwriting file.
File saved as `delaware_veh_registrations.csv`
File `district of columbia_veh_registrations.c

C:\Users\chris\AppData\Local\Temp\ipykernel_33612\3137593107.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  states_df.sort_values(by=["year", "electric", "gasoline"], inplace=True)
